In [1]:
!nvidia-smi

Thu Sep 26 10:01:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.6     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   28C    P0              62W / 400W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install tensorflow

In [ ]:
!python3 -m pip install tensorflow[and-cuda]
# Verify the installation:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

In [3]:
import tensorflow as tf


# Create a MirroredStrategy with GPU devices
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3", "/gpu:4", "/gpu:5", "/gpu:6", "/gpu:7"])


print("Number of devices: {}".format(mirrored_strategy.num_replicas_in_sync))



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
Number of devices: 8


In [4]:
import keras
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model


def get_dataset():
    batch_size = 32
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a `tf.data.Dataset`.
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=20, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/20
INFO:tensorflow:Collective all_reduce tensors: 6 all_reduces, num_devices = 8, group_size = 8, implementation = CommunicationImplementation.NCCL, num_packs = 1
1556/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3798 - sparse_categorical_accuracy: 0.8897

2024-09-26 10:02:30.566133: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-09-26 10:02:30.566198: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-09-26 10:02:30.566258: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-09-26 10:02:30.566324: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-09-26 10:02:30.566690: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorG

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 0.3790 - sparse_categorical_accuracy: 0.8900 - val_loss: 0.1446 - val_sparse_categorical_accuracy: 0.9584
Epoch 2/20


2024-09-26 10:02:34.232172: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


1556/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1053 - sparse_categorical_accuracy: 0.9688INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 0.1053 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.1162 - val_sparse_categorical_accuracy: 0.9648
Epoch 3/20


...

2024-09-26 10:04:10.649505: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0159 - sparse_categorical_accuracy: 0.9949 - val_loss: 0.1390 - val_sparse_categorical_accuracy: 0.9712
Epoch 10/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0150 - sparse_categorical_accuracy: 0.9953 - val_loss: 0.1419 - val_sparse_categorical_accuracy: 0.9744
Epoch 11/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0161 - sparse_categorical_accuracy: 0.9943 - val_loss: 0.1725 - val_sparse_categorical_accuracy: 0.9696
Epoch 12/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.0157 - sparse_categorical_accuracy: 0.9951 - val_loss: 0.1493 - val_sparse_categorical_accuracy: 0.9736
Epoch 13/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.0135 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.1856 - val_sparse_categorical_accuracy: 0.9696
Epoch 14/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0115 - sparse_categorical_accuracy: 0.9964 - val_loss: 0.1561 - val_sparse_categorical_accuracy:

2024-09-26 10:05:49.828466: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9965 - val_loss: 0.1775 - val_sparse_categorical_accuracy: 0.9752
Epoch 18/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.0101 - sparse_categorical_accuracy: 0.9973 - val_loss: 0.1894 - val_sparse_categorical_accuracy: 0.9720
Epoch 19/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 0.0127 - sparse_categorical_accuracy: 0.9960 - val_loss: 0.1968 - val_sparse_categorical_accuracy: 0.9752
Epoch 20/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.0078 - sparse_categorical_accuracy: 0.9974 - val_loss: 0.2501 - val_sparse_categorical_accuracy: 0.9736
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1905 - sparse_categorical_accuracy: 0.9728


[0.12842562794685364, 0.9824000000953674]

In [ ]:
import tensorflow as tf
import gc


# Clear GPU memory
tf.keras.backend.clear_session()
del model
gc.collect()